In [ ]:
import os
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

In [ ]:
# Step 0 - Initialize variables

width = 224
height = 224
nb_epochs = 5
trainingPath = 'img/train'
validationPath = "img/valid"
testPath = "img/test"
nb_classes = len(next(os.walk(trainingPath))[1])

In [ ]:
# Step 1 - Load data

datagen = ImageDataGenerator(rescale = 1./255,
                             rotation_range = 20,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             horizontal_flip = True,
                             fill_mode='nearest')

training = datagen.flow_from_directory(trainingPath,
                                       target_size = (width, height),
                                       batch_size = 10,
                                       class_mode = 'categorical')
validation = datagen.flow_from_directory(validationPath,
                                         target_size = (width, height),
                                         batch_size = 4,
                                         class_mode = 'categorical')

nb_classes = len(next(os.walk(trainingPath))[1])

In [ ]:
# Step 2 - Load the VGG model

vgg_conv = VGG16(weights = 'imagenet', include_top = False, input_shape = (width, height, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [ ]:
# Step 2 - Build a model from VGG16

# Create the model
model = models.Sequential()
 
# Add the VGG convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(nb_classes, activation = 'softmax'))

In [ ]:
# Step 3 - Compile model

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.RMSprop(lr = 1e-4),
              metrics = ['acc'])

In [ ]:
# Step 4 - Train model

model.fit_generator(training,
                    steps_per_epoch = len(training),
                    epochs = nb_epochs,
                    validation_data = validation,
                    validation_steps = len(validation),
                    verbose = 2)

model.save_weights('model/vgg16.h5')

In [ ]:
# Step 5 - Evaluate the performance with the test set

test = datagen.flow_from_directory(testPath,
                                   target_size = (width, height),
                                   batch_size = 10,
                                   class_mode = 'categorical')

predictions = model.predict_generator(test,
                                      steps = len(test),
                                      verbose = 2)